In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Schizophrenia"
cohort = "GSE145554"

# Input paths
in_trait_dir = "../../input/GEO/Schizophrenia"
in_cohort_dir = "../../input/GEO/Schizophrenia/GSE145554"

# Output paths
out_data_file = "../../output/preprocess/Schizophrenia/GSE145554.csv"
out_gene_data_file = "../../output/preprocess/Schizophrenia/gene_data/GSE145554.csv"
out_clinical_data_file = "../../output/preprocess/Schizophrenia/clinical_data/GSE145554.csv"
json_path = "../../output/preprocess/Schizophrenia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional profile of pyramidal neurons in chronic schizophrenia reveals lamina-specific dysfunction of neuronal immunity"
!Series_summary	"While the pathophysiology of schizophrenia has been extensively investigated using homogenized postmortem brain samples, few studies have examined changes in brain samples with techniques that may attribute perturbations to specific cell types. To fill this gap, we performed microarray assays on mRNA isolated from anterior cingulate cortex (ACC) superficial and deep pyramidal neurons from 12 schizophrenia and 12 control subjects using laser capture microdissection. Among all the annotated genes, we identified 134 significantly increased and 130 decreased genes in superficial pyramidal neurons, while 93 significantly increased and 101 decreased genes were found in deep pyramidal neurons, in schizophrenia compared to control subjects. In these differentially expressed genes, we detected lamina-specific chan

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Check if gene expression data is available
is_gene_available = True  # Based on the background info, this dataset contains gene expression data from microarray assays

# 2.1 Identify keys for trait, age, and gender in the sample characteristics dictionary
trait_row = 0  # "disease state" contains schizophrenia vs control
age_row = 3    # "age" information is available
gender_row = 1  # "Sex" information is available

# 2.2 Data type conversion functions for each variable

def convert_trait(value):
    """Convert trait values to binary 0/1 format."""
    if value is None:
        return None
    # Extract value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if "schizophrenia" in value:
        return 1
    elif "control" in value:
        return 0
    else:
        return None

def convert_age(value):
    """Convert age values to numeric format."""
    if value is None:
        return None
    # Extract value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    else:
        value = value.strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender values to binary 0/1 format (female=0, male=1)."""
    if value is None:
        return None
    # Extract value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    else:
        return None

# 3. Save metadata - initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical feature extraction (if trait data is available)
if trait_row is not None:
    # Create the sample characteristics DataFrame from the dictionary provided in the previous step
    sample_characteristics_dict = {
        0: ['disease state: schizophrenia', 'disease state: control'], 
        1: ['Sex: Male', 'Sex: Female'], 
        2: ['pmi: 372', 'pmi: 250', 'pmi: 1225', 'pmi: 255', 'pmi: 203', 'pmi: 1110', 'pmi: 504', 'pmi: 600', 'pmi: 230', 'pmi: 1140', 'pmi: 940', 'pmi: 417', 'pmi: 454', 'pmi: 1092', 'pmi: 410', 'pmi: 583', 'pmi: 534', 'pmi: 737', 'pmi: 465', 'pmi: 720', 'pmi: 415', 'pmi: 800', 'pmi: 1220'], 
        3: ['age: 63', 'age: 90', 'age: 83', 'age: 69', 'age: 73', 'age: 84', 'age: 66', 'age: 78', 'age: 65', 'age: 89', 'age: 74', 'age: 86', 'age: 79', 'age: 77', 'age: 68', 'age: 59', 'age: 58', 'age: 72', 'age: 80', 'age: 57'], 
        4: ['ph: 5.9', 'ph: 6', 'ph: 7.1', 'ph: 6.3', 'ph: 6.21', 'ph: 6.69', 'ph: 6.19', 'ph: 6.82', 'ph: 6.72', 'ph: 6.68', 'ph: 6.58', 'ph: 5.8', 'ph: 6.6', 'ph: 6.01', 'ph: 6.27', 'ph: 6.63', 'ph: 6.17', 'ph: 5.97', 'ph: 6.9', 'ph: 6.1'], 
        5: ['meds: ON', 'meds: OFF', 'meds: Unknown'], 
        6: ['tissue: pyramidal neurons'], 
        7: ['tissue source: Neurons captured by LCM isolated from superficial (lamina II-III) layers', 'tissue source: Neurons captured by LCM isolated from deep (lamina V-VI)  layers']
    }
    
    # Convert to DataFrame format expected by geo_select_clinical_features
    clinical_data = pd.DataFrame.from_dict(sample_characteristics_dict, orient='index')
    
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(clinical_df)
    print("Preview of clinical data:")
    print(preview)
    
    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{0: [1.0, 63.0, 1.0], 1: [0.0, 90.0, 0.0], 2: [nan, 83.0, nan], 3: [nan, 69.0, nan], 4: [nan, 73.0, nan], 5: [nan, 84.0, nan], 6: [nan, 66.0, nan], 7: [nan, 78.0, nan], 8: [nan, 65.0, nan], 9: [nan, 89.0, nan], 10: [nan, 74.0, nan], 11: [nan, 86.0, nan], 12: [nan, 79.0, nan], 13: [nan, 77.0, nan], 14: [nan, 68.0, nan], 15: [nan, 59.0, nan], 16: [nan, 58.0, nan], 17: [nan, 72.0, nan], 18: [nan, 80.0, nan], 19: [nan, 57.0, nan], 20: [nan, nan, nan], 21: [nan, nan, nan], 22: [nan, nan, nan]}
Clinical data saved to ../../output/preprocess/Schizophrenia/clinical_data/GSE145554.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Schizophrenia/GSE145554/GSE145554_series_matrix.txt.gz
Gene data shape: (32321, 43)
First 20 gene/probe identifiers:
Index(['7892501', '7892502', '7892503', '7892504', '7892505', '7892506',
       '7892507', '7892508', '7892509', '7892510', '7892511', '7892512',
       '7892513', '7892514', '7892515', '7892516', '7892517', '7892518',
       '7892519', '7892520'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers
# These appear to be probe IDs (numeric identifiers) rather than human gene symbols
# Human gene symbols typically have alphabetic characters (like BRCA1, TP53, etc.)
# These numeric identifiers (7892501, 7892502, etc.) are likely probe IDs from a microarray platform
# and need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the SPOT_ID.1 column which appears to contain gene names
print("\nAnalyzing SPOT_ID.1 column for gene symbols:")
if 'SPOT_ID.1' in gene_annotation.columns:
    # Extract a few sample values
    sample_values = gene_annotation['SPOT_ID.1'].head(3).tolist()
    for i, value in enumerate(sample_values):
        print(f"Sample {i+1} excerpt: {value[:200]}...")  # Print first 200 chars
        # Test the extract_human_gene_symbols function on these values
        symbols = extract_human_gene_symbols(value)
        print(f"  Extracted gene symbols: {symbols}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0].split('_')[0]  # Get prefix of first gene ID
print(f"\nGene data ID prefix: {gene_data_id_prefix}")

# Look for columns that might match the gene data IDs
for col in gene_annotation.columns:
    if gene_annotation[col].astype(str).str.contains(gene_data_id_prefix).any():
        print(f"Column '{col}' contains values matching gene data ID pattern")

# Check if there's any column that might contain transcript or gene IDs
print("\nChecking for columns containing transcript or gene related terms:")
for col in gene_annotation.columns:
    if any(term in col.upper() for term in ['GENE', 'TRANSCRIPT', 'SYMBOL', 'NAME', 'DESCRIPTION']):
        print(f"Column '{col}' may contain gene-related information")
        # Show sample values
        print(f"Sample values: {gene_annotation[col].head(3).tolist()}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_LIST', 'SPOT_ID', 'seqname', 'RANGE_GB', 'RANGE_STRAND', 'RANGE_START', 'RANGE_STOP', 'total_probes', 'gene_assignment', 'mrna_assignment', 'category']
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908', 'NR_024437,XM_006711854,XM_006726377,XR_430662,AK298283,AL137655,BC032332,BC118988,BC122537,BC131690,NM_207366,AK301928,BC071667', 'NM_001005221,NM_001005224,NM_001005277,NM_001005504,BC137547,BC137568'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091', '334129', '367659'], 'RANGE_STOP': ['54936', '63887', '70008', '

Column 'ID' contains values matching gene data ID pattern



Checking for columns containing transcript or gene related terms:
Column 'seqname' may contain gene-related information
Sample values: ['chr1', 'chr1', 'chr1']
Column 'gene_assignment' may contain gene-related information
Sample values: ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // --- // --- /// ENST00000492842 // OR4G11P // olfactory receptor, family 4, subfamily G, member 11 pseudogene // --- // --- /// ENST00000588632 // OR4G1P // olfactory receptor, family 4, subfamily G, member 1 pseudogene // --- // ---', 'NM_001004195 // OR4F4 // olfactory receptor, family 4, subfamily F, member 4 // 15q26.3 // 26682 /// NM_001005240 // OR4F17 // olfactory receptor, family 4, subfamily F, member 17 // 19p13.3 // 81099 /// NM_001005484 // OR4F5 // olfactory receptor, family 4, subfamily F, member 5 // 1p36.33 // 79501 /// ENST00000318050 // OR4F17 // olfactory receptor, family 4, subfamily F, member 17 // 19p13.3 // 81099 /// ENST0000032

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns from gene_annotation
# The 'ID' column in gene_annotation matches the index of gene_data
# The 'gene_assignment' column contains gene symbols information

# 2. Create a mapping dataframe from ID to gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("Sample of mapping dataframe:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probes to gene expressions
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 5 genes and sample counts:")
print(gene_data.head())

# Save the processed gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")


Mapping dataframe shape: (33297, 2)
Sample of mapping dataframe:
        ID                                               Gene
0  7896736                                                ---
1  7896738  ENST00000328113 // OR4G2P // olfactory recepto...
2  7896740  NM_001004195 // OR4F4 // olfactory receptor, f...
3  7896742  NR_024437 // LOC728323 // uncharacterized LOC7...
4  7896744  NM_001005221 // OR4F29 // olfactory receptor, ...


Gene expression data shape after mapping: (117447, 43)
First 5 genes and sample counts:
      GSM4321410  GSM4321411  GSM4321412  GSM4321413  GSM4321414  GSM4321415  \
Gene                                                                           
A-     26.855859   27.203828   26.176372   26.485634   27.129572   26.785776   
A-3-    0.676297    0.827000    0.786592    0.700897    0.691317    0.688663   
A-52    0.763744    0.822074    0.841448    0.893042    0.798220    0.812496   
A-E     1.058217    1.061592    1.057221    1.097058    1.119544    0.977072   
A-I     1.528381    1.560647    1.581738    1.504029    1.412246    1.480298   

      GSM4321416  GSM4321417  GSM4321418  GSM4321419  ...  GSM4321443  \
Gene                                                  ...               
A-     26.380192   26.041538   26.106642   27.330138  ...   27.254018   
A-3-    0.700579    0.661088    0.768229    0.693318  ...    0.724141   
A-52    0.848997    0.801966    0.829038    0.783488  ...  

Gene data saved to ../../output/preprocess/Schizophrenia/gene_data/GSE145554.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data to file
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error during gene normalization: {e}")
    # Continue with original gene data if normalization fails
    normalized_gene_data = gene_data
    print("Using original gene data without normalization")

# 2. Link the clinical and genetic data
# First, fix the clinical data format and properly transpose it
clinical_df = pd.read_csv(out_clinical_data_file)
print(f"Loaded clinical data shape: {clinical_df.shape}")
print("Clinical data preview:")
print(clinical_df.head())

# Set proper index for clinical data (each row represents a feature)
clinical_df.index = ["Schizophrenia", "Age", "Gender"]
print(f"Clinical data with proper index:")
print(clinical_df.head())

# Transpose to get samples as rows and features as columns
clinical_df_t = clinical_df.T
print(f"Transposed clinical data shape: {clinical_df_t.shape}")
print("Sample clinical values after transposition:")
print(clinical_df_t.head())

# Count valid clinical data to diagnose issues
valid_trait_count = clinical_df_t['Schizophrenia'].notna().sum()
valid_age_count = clinical_df_t['Age'].notna().sum()
valid_gender_count = clinical_df_t['Gender'].notna().sum()

print(f"Valid clinical data counts - Schizophrenia: {valid_trait_count}, Age: {valid_age_count}, Gender: {valid_gender_count}")

# Check if we have sufficient clinical data to proceed
if valid_trait_count < 5:
    print(f"WARNING: Very few samples ({valid_trait_count}) have valid trait data. Dataset may not be usable.")

# Link the clinical and genetic data
try:
    # Ensure column names are consistent between datasets
    # Convert sample IDs to the format used in gene data (GSM numbers)
    gene_samples = normalized_gene_data.columns
    clinical_samples = clinical_df_t.index
    
    print(f"Gene data sample format: {gene_samples[:3]}")
    print(f"Clinical data indices: {clinical_samples[:3]}")
    
    # Match sample formats between datasets if needed
    # For this dataset, just use the samples common to both datasets
    common_samples = list(set(gene_samples).intersection(set([f"GSM{idx+4321410}" for idx in range(len(clinical_samples))])))
    
    if not common_samples:
        print("No direct sample name matches. Attempting to align by position...")
        # Match by position if names don't match directly
        if len(gene_samples) >= len(clinical_samples):
            # Rename clinical data indices to match gene data column names
            clinical_df_t.index = gene_samples[:len(clinical_samples)]
            common_samples = list(clinical_df_t.index)
        else:
            print("Cannot align samples - unequal lengths")
            common_samples = []
    
    print(f"Common samples between clinical and genetic data: {len(common_samples)}")
    
    if len(common_samples) >= 5:
        # Link the data
        linked_data = pd.DataFrame(index=common_samples)
        
        # Add clinical features
        for feature in clinical_df_t.columns:
            if feature in clinical_df_t.columns:
                linked_data[feature] = clinical_df_t.loc[common_samples, feature].values
        
        # Add gene features
        for gene in normalized_gene_data.index:
            linked_data[gene] = normalized_gene_data.loc[gene, common_samples].values
            
        # Reset index to get sample IDs as a column
        linked_data = linked_data.reset_index().rename(columns={'index': 'Sample_ID'})
        
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 3. Handle missing values
        print("Handling missing values...")
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        if linked_data.shape[0] > 0:
            # 4. Check for bias in features
            is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
            print(f"Data shape after removing biased features: {linked_data.shape}")
            
            # 5. Validate and save cohort information
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=is_biased,
                df=linked_data,
                note="Gene expression from pyramidal neurons in anterior cingulate cortex of schizophrenia patients."
            )
            
            # 6. Save the linked data if usable
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data.to_csv(out_data_file, index=False)
                print(f"Linked data saved to {out_data_file}")
            else:
                print("Dataset is not usable for analysis. No linked data file saved.")
        else:
            print("No valid samples remaining after handling missing values. Dataset is not usable.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=False,
                is_biased=True,
                df=pd.DataFrame(),
                note="Insufficient clinical-genetic correspondence. No usable samples after filtering."
            )
    else:
        print("Insufficient common samples between clinical and genetic data.")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=True,
            df=pd.DataFrame(),
            note="Insufficient overlap between clinical and genetic data samples."
        )
except Exception as e:
    print(f"Error during data linking or processing: {e}")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=True,
        df=pd.DataFrame(),
        note=f"Error during processing: {str(e)}"
    )

Gene data shape after normalization: (24221, 43)


Normalized gene expression data saved to ../../output/preprocess/Schizophrenia/gene_data/GSE145554.csv
Loaded clinical data shape: (3, 23)
Clinical data preview:
      0     1     2     3     4     5     6     7     8     9  ...    13  \
0   1.0   0.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   NaN   
1  63.0  90.0  83.0  69.0  73.0  84.0  66.0  78.0  65.0  89.0  ...  77.0   
2   1.0   0.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   NaN   

     14    15    16    17    18    19  20  21  22  
0   NaN   NaN   NaN   NaN   NaN   NaN NaN NaN NaN  
1  68.0  59.0  58.0  72.0  80.0  57.0 NaN NaN NaN  
2   NaN   NaN   NaN   NaN   NaN   NaN NaN NaN NaN  

[3 rows x 23 columns]
Clinical data with proper index:
                  0     1     2     3     4     5     6     7     8     9  \
Schizophrenia   1.0   0.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
Age            63.0  90.0  83.0  69.0  73.0  84.0  66.0  78.0  65.0  89.0   
Gender          1.0   0.0   NaN   NaN 